<a href="https://colab.research.google.com/github/KidsareBornStars/Study/blob/main/Balls_Classification_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/balls

/content/drive/MyDrive/balls


In [ ]:
# Importing Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn 
import torch.utils.data as data
from torchvision import transforms, datasets, models
import torch.optim as optim
from torchsummary import summary

In [ ]:
# CUDA error: device-side assert triggered CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect. For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
# Debugging을 위한 Code
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Transfer Learning시에 fully-connected layer의 dimension을 잘못 맞추어 일어난 오류.

In [ ]:
base_dir = "/content/drive/MyDrive/balls"

train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")
valid_dir = os.path.join(base_dir,"valid")

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((128,128))]) # 1~255의 rgb를 0~1로 normalize + classes보면 있는 사이즈 그대로 맞추기

In [ ]:
train_data = datasets.ImageFolder(root=train_dir, transform = transform)
test_data = datasets.ImageFolder(root=test_dir, transform = transform)
valid_data = datasets.ImageFolder(root=valid_dir, transform = transform)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # cpu환경에서의 학습이 느려서 gpu환경에서 학습

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
train_loader = data.DataLoader(train_data,64,True)
test_loader = data.DataLoader(test_data,64,False)
valid_loader = data.DataLoader(valid_data,64,False)

In [ ]:
def train(epoch):
    model.train()

    train_loss = 0
    correct = 0
    for i, (input,label) in enumerate(train_loader):
        input=input.to(device)
        label=label.to(device)
        output = model(input)
        loss = criterion(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (output.argmax(dim=1) == label).type(torch.float).sum().item()

        if (i+1)%10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)] \tLoss: {:.3f} \tAccuracy: {:.2f}%'.format(
            epoch+1,
            (i+1) * len(input),
            len(train_loader.dataset),
            100. * (i+1) / len(train_loader),
            loss.item(),
            100. * correct / ((i+1) * len(input))))

In [ ]:
def test(data_loader):
    model.eval()

    test_loss =0
    correct = 0
    for input,label in data_loader:
        input,label = input.to(device),label.to(device)
        output = model(input)

        test_loss += criterion(output,label).item()
        correct += (output.argmax(dim=1)==label).type(torch.float).sum().item()

    test_loss /= len(test_loader)
    accuracy  = 100. * correct / len(valid_loader.dataset)

    print(f"\nTest set: Average loss: {test_loss:.2f}, Accuracy: {accuracy:.0f}%\n") 

In [ ]:
criterion=nn.CrossEntropyLoss()

In [ ]:
# torchvision.models의 resnet18을 사용해 보았다. 
# Label이 26개로 많기 때문에 feature extractor에 backpropagation
# Accuracy가 90%이상으로 매우 높지만, Overfitting이 된 것으로 보이는 epoch이 여럿 존재한다
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 26) # 26개의 feature로 반환시킨다
optimizer = optim.Adam(model.parameters(),lr=1e-3)
model.to(device)
for epoch in range(15):
    train(epoch)
    test(valid_loader)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Train Epoch: 1 [640/3051 (21%)] 	Loss: 1.053 	Accuracy: 62.97%
Train Epoch: 1 [1280/3051 (42%)] 	Loss: 0.805 	Accuracy: 71.25%
Train Epoch: 1 [1920/3051 (62%)] 	Loss: 0.626 	Accuracy: 75.05%
Train Epoch: 1 [2560/3051 (83%)] 	Loss: 1.183 	Accuracy: 75.82%

Test set: Average loss: 0.71, Accuracy: 80%

Train Epoch: 2 [640/3051 (21%)] 	Loss: 0.409 	Accuracy: 89.38%
Train Epoch: 2 [1280/3051 (42%)] 	Loss: 0.308 	Accuracy: 90.23%
Train Epoch: 2 [1920/3051 (62%)] 	Loss: 0.230 	Accuracy: 90.05%
Train Epoch: 2 [2560/3051 (83%)] 	Loss: 0.478 	Accuracy: 90.35%

Test set: Average loss: 0.28, Accuracy: 91%

Train Epoch: 3 [640/3051 (21%)] 	Loss: 0.205 	Accuracy: 92.03%
Train Epoch: 3 [1280/3051 (42%)] 	Loss: 0.195 	Accuracy: 92.03%
Train Epoch: 3 [1920/3051 (62%)] 	Loss: 0.107 	Accuracy: 92.08%
Train Epoch: 3 [2560/3051 (83%)] 	Loss: 0.196 	Accuracy: 92.11%

Test set: Average loss: 0.24, Accuracy: 89%

Train Epoch: 4 [640/3051 (21%)] 	Loss: 0.161 	Accuracy: 94.69%
Train Epoch: 4 [1280/3051 (42%)] 	

In [13]:
# Feature extractor에 대해 backpropagation을 하지않는 Frozen기법 사용
# Overfitting이 방지되었고, 정확도도 매우 높다
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 26)
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(15):
    train(epoch)
    test(valid_loader)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Epoch: 1 [640/3051 (21%)] 	Loss: 2.658 	Accuracy: 15.94%
Train Epoch: 1 [1280/3051 (42%)] 	Loss: 1.870 	Accuracy: 30.94%
Train Epoch: 1 [1920/3051 (62%)] 	Loss: 1.428 	Accuracy: 43.91%
Train Epoch: 1 [2560/3051 (83%)] 	Loss: 1.130 	Accuracy: 51.60%

Test set: Average loss: 0.97, Accuracy: 80%

Train Epoch: 2 [640/3051 (21%)] 	Loss: 0.928 	Accuracy: 82.81%
Train Epoch: 2 [1280/3051 (42%)] 	Loss: 0.664 	Accuracy: 85.55%
Train Epoch: 2 [1920/3051 (62%)] 	Loss: 0.621 	Accuracy: 85.36%
Train Epoch: 2 [2560/3051 (83%)] 	Loss: 0.737 	Accuracy: 85.66%

Test set: Average loss: 0.60, Accuracy: 86%

Train Epoch: 3 [640/3051 (21%)] 	Loss: 0.366 	Accuracy: 90.78%
Train Epoch: 3 [1280/3051 (42%)] 	Loss: 0.458 	Accuracy: 90.55%
Train Epoch: 3 [1920/3051 (62%)] 	Loss: 0.541 	Accuracy: 90.57%
Train Epoch: 3 [2560/3051 (83%)] 	Loss: 0.460 	Accuracy: 90.43%

Test set: Average loss: 0.48, Accuracy: 88%

Train Epoch: 4 [640/3051 (21%)] 	Loss: 0.356 	Accuracy: 94.06%
Train Epoch: 4 [1280/3051 (42%)] 	

In [ ]:
# Pretrained된 Model로 Transfer Learning을 하는 것이 정확도가 매우 높다